# Crop reallocation algorithm: toy model (numpy version)
Trying this one more time, this time using numpy instead of xarray. Need for speed...

5/1/20: Adding chi calculation to this.

In [17]:
import numpy as np
import pandas as pd
import xarray as xr
import timeit
import copy
from numba import jit

# 1. Set up parameters
Use data from Ishan's code, as numpy arrays

In [106]:
# define array dims
crops = np.array(['soy', 'rice'])
geo0 = np.array([1])
geo1 = np.array([1, 2, 3])
total_acres = np.array([100, 100, 100])

In [107]:
soy_calories_per_bushel = 25
rice_calories_per_bushel = 15

In [108]:
# present data
present_soy_yields = np.array([10, 20, 15])
present_rice_yields = np.array([20, 10, 15])

# what happens if you scale by a linear factor?
present_soy_yields = present_soy_yields
present_rice_yields = present_rice_yields

present_soy_acreage = np.array([40, 70, 0])
present_rice_acreage = np.array([60, 30, 0])

present_soy_calories = present_soy_yields*soy_calories_per_bushel
present_rice_calories = present_rice_yields*rice_calories_per_bushel

present_total_planted_acreage = present_soy_acreage + present_rice_acreage

In [109]:
# future data
future_soy_yield_shocks = np.array([0.5, 0.8, 1])
future_rice_yield_shocks = np.array([0.9, 0.6, 1])

future_soy_yields = present_soy_yields * future_soy_yield_shocks
future_rice_yields = present_rice_yields * future_rice_yield_shocks

future_soy_acreage = present_soy_acreage
future_rice_acreage = present_rice_acreage

future_soy_calories = future_soy_yields*soy_calories_per_bushel
future_rice_calories = future_rice_yields*rice_calories_per_bushel

# 2. Set up functions for calculating moments

In [22]:
def calculate_gamma(args):
    '''
    Docstring here!
    Parameters:
    -----------
    args: list
        A list with one element per crop. Each element of the list should be another list
        of length 2, where the first element is an array of the acres planted of that crop
        and the second is an array of the yields in calories per bushel of that crop.
    '''
    # calculate the number of calories produced in each plot for each crop,
    # the sum them all up
    total_cal = sum([sum(acres*cals) for acres, cals in args])

    # calculate potential calories by picking the max calorie yield on each
    # plot and applying it to all the acres planted for that crop
    max_cal = np.amax(np.stack([args[i][1] for i in range(len(args))]), axis=0)
    total_acres = np.sum(np.stack([args[i][0] for i in range(len(args))]), axis=0)
    potential_cal = sum(max_cal * total_acres)

    return total_cal / potential_cal

In [23]:
def analyze_empty_acreage(total_acres, acres_planted, yields, crop_acreage):
    '''
    Docstring here!
    '''
    empty_acres = total_acres - acres_planted

    assert(all(empty_acres >= 0))

    empty_max_yield = yields[empty_acres > 0].max()
    empty_max_id = np.argwhere(
        (empty_acres > 0) & (yields == empty_max_yield)).item()

    used_min_yield = yields[acres_planted > 0].min()
    used_min_id = np.argwhere(
        (crop_acreage > 0) & (yields == used_min_yield))
    
    if used_min_id.size == 0:
        used_min_id = np.nan
    else:
        used_min_id = used_min_id.item()

    return [empty_max_id, used_min_id]

In [24]:
def calculate_phi(total_acres, acres_planted, yields, crop_acreage):
    '''
    docstring here!
    '''
    # calculate actual yield
    actual_yield = (crop_acreage*yields).sum()

    # calculate potential yield
    empty_max_id, used_min_id = (
        analyze_empty_acreage(total_acres, acres_planted, yields, crop_acreage))
    
    crop_acreage = crop_acreage.copy()
    acres_planted = acres_planted.copy()
    
    while (not np.isnan(used_min_id)) and yields[empty_max_id] > yields[used_min_id]:
        for a in crop_acreage, acres_planted:
            a[empty_max_id] += 1
            a[used_min_id] -= 1

        empty_max_id, used_min_id = (
            analyze_empty_acreage(total_acres, acres_planted, yields, crop_acreage))
    
    potential_yield = (crop_acreage*yields).sum()

    return actual_yield/potential_yield

# 3. Write functions that incorporate a climate shock and match moments

In [110]:
present_both_args = [total_acres, present_total_planted_acreage]
present_soy_args = present_both_args + [present_soy_yields, present_soy_acreage, present_soy_calories]
present_rice_args = present_both_args + [present_rice_yields, present_rice_acreage, present_rice_calories]
present_args = [present_soy_args, present_rice_args]

future_both_args = [total_acres, present_total_planted_acreage]
future_soy_args = future_both_args + [future_soy_yields, future_soy_acreage, future_soy_calories]
future_rice_args = future_both_args + [future_rice_yields, future_rice_acreage, future_rice_calories]
future_args = [future_soy_args, future_rice_args]

In [26]:
def calculate_distances(present_args, future_args):
    '''
    docstring here!
    '''
    present_moments = (
       [calculate_phi(a[0], a[1], a[2], a[3]) for a in present_args] +  
       [calculate_gamma([present_args[i][3:] for i in range(len(present_args))])]
    )
    
    future_moments = (
       [calculate_phi(a[0], a[1], a[2], a[3]) for a in future_args] +  
       [calculate_gamma([future_args[i][3:] for i in range(len(future_args))])]
    )

    distances = [p - f for p, f in zip(present_moments, future_moments)]

    return distances

In [119]:
step_size = 1
def match_moments(present_args, future_args, step_size=step_size):
    '''
    Given data on two periods, modify the values in the second period until
    moments match across the two periods.

    Parameters
    ----------
    Same as calculate_distances
    Returns
    -------
    The modified data for the second period.
    '''
    # logger.debug('Calculating initial set of distances')
    distances = calculate_distances(present_args, future_args)

    future_args = copy.deepcopy(future_args)
    present_args = copy.deepcopy(present_args)

    counter = 0

    # logger.debug('Beginning initial distance loop')
    while any(d > 0 for d in distances):
        reallocation_info = [analyze_empty_acreage(a[0], a[1], a[2], a[3])
                             for a in future_args]
        # phi iteration happens first
        # note this relies on the fact that calculate_distances returns gamma
        # last.
        # logger.debug('Iterate phi')
        if any(d > 0 for d in distances[:-1]):
            for i in range(len(distances) - 1):
                if distances[i] > 0:
                    for a in [future_args[i][1], future_args[i][3]]:
                        a[reallocation_info[i][0]] += step_size  # empty_max_id
                        a[reallocation_info[i][1]] -= step_size   # used_min_id

        # now iterate over gamma
        # logger.debug('Iterate gamma')
        if any(d > 0 for d in distances[-1:]):
            # switch an acre between the lowest-yielding and highest-yielding
            # crop in the location where that gap is largest
            # create list of crop level yields from args
            crop_yield_list = [future_args[i][2]
                               for i in range(len(future_args))]

            diffs = []
            for i in range(len(crop_yield_list)):
                j_list = list(crop_yield_list)  # modify a new copy
                del j_list[i]
                i_val = crop_yield_list[i]
                i_list = [i_val, i_val]
                if range(len(j_list) > 2):
                    for i in range(len(j_list)):
                        i_list = [i_val, i_list]
                diffs += [(i - j).tolist() for i, j in zip(i_list, j_list)]

            # flatten the list of diffs and take absolute values
            diffs = [i for diff in diffs for i in diff]
            # note this is picking the first one--not sure if this is desirable
            max_diff_id = diffs.index(max(diffs))
            min_diff_id = diffs.index(min(diffs))

            # check there are equal numbers of plots for each crop
            test_list = [len(crop_yield_list[i])
                         for i in range(len(crop_yield_list))]
            assert all(
                [test_list[i] == test_list[0] for i in range(len(test_list))]
            )
            n_plots = test_list[0]
            max_crop_id, min_crop_id = [
                i // (n_plots * (len(crop_yield_list) - 1))
                for i in [max_diff_id, min_diff_id]]

            max_plot_id, min_plot_id = [i % n_plots
                                        for i in [max_diff_id, min_diff_id]
                                        ]
            assert min_plot_id == max_plot_id

            future_args[min_crop_id][3][min_plot_id] -= step_size
            future_args[max_crop_id][3][max_plot_id] += step_size

        # logger.debug('Re-calculate distances')
        distances = calculate_distances(present_args, future_args)
        counter += 1
        
        # return future_args
        # logger.debug('Distances are {} on interation {}.'
        #              .format(distances, counter))

        # calculate Q1 and return it
        Q1 = sum([future_args[i][3] * future_args[i][4]
                 for i in range(len(future_args))]).sum()
    return Q1

In [77]:
def maximize_output(args, total_area, total_planted_area):
    '''
    Calculate the maximum possible output given total planted area and yields
    in each region.
    Parameters
    ----------
    args: list
        A list of lists. Each sub-list should be another list, corresponding to
        the data representing one of the two periods we want to compare. The
        elements of the sub-sub-lists should be arrays, where each set of
        arrays corresponds to a particular crop. Each set of arrays should be
        of the same length, and should be ordered such that corresponding
        indices of different arrays refer to the same region.
        The elements of the list should be as follows: total arable acres in
        each region, total planted acres, crop yields per hectare,
        crop-specific acreage, and calories yielded per hectare.
    total_area: array
        Array of the total number of arable hectares in each region. Should be
        sorted so indices correspond to the indices in args.
    total_planted_area: array
        Array of the total number of hectares planted in each region. Should be
        sorted so indices correspond to the indices in args.

    '''
    max_yields = np.amax([args[i][4] for i in range(len(args))], axis=0)

    df = (pd.DataFrame({'max_yield': max_yields, 'area': total_area})
            .sort_values(by='max_yield', ascending=False))
    df['cumulative_area'] = df.area.cumsum()

    total_planted_area = sum(total_planted_area)

    # identify the top-yielding regions
    max_yield_regions = df.loc[df.cumulative_area <= total_planted_area]
    # add the first region that is outside of the top, planting only as many
    # hectares as are left over
    residual_region = df.loc[df.cumulative_area > total_planted_area].iloc[[0]]
    residual_space = total_planted_area - sum(max_yield_regions.area)
    residual_region.area = residual_space

    max_yield_regions = pd.concat([max_yield_regions, residual_region], axis=0)

    max_yield = sum(max_yield_regions.area * max_yield_regions.max_yield)
    return max_yield


def calculate_chi(args, total_area, total_planted_area):
    '''
    Calculate chi, the ratio of calories grown to the maximum possible
    number of calories grown if crops were chosen and allocated perfectly,
    conditional on the total number of acres planted.
    Parameters
    ----------
    args: list
        A list of lists. Each sub-list should be another list, corresponding to
        the data representing one of the two periods we want to compare. The
        elements of the sub-sub-lists should be arrays, where each set of
        arrays corresponds to a particular crop. Each set of arrays should be
        of the same length, and should be ordered such that corresponding
        indices of different arrays refer to the same region.
        The elements of the list should be as follows: total arable acres in
        each region, total planted acres, crop yields per hectare,
        crop-specific acreage, and calories yielded per hectare.
    total_area: array
        Array of the total number of arable hectares in each region. Should be
        sorted so indices correspond to the indices in args.
    total_planted_area: array
        Array of the total number of hectares planted in each region. Should be
        sorted so indices correspond to the indices in args.
    '''
    # sum up total calories by region
    calories_grown = sum([args[i][3] * args[i][4] for i in range(len(args))])
    # sum over all the regions--this is the numerator of chi
    calories_grown = calories_grown.sum()

    max_yield = maximize_output(args, total_area, total_planted_area)

    return calories_grown / max_yield


def match_moment(present_args, future_args, total_area, total_planted_area):
    '''
    Calculate Q1, the number of calories produced in a future period.
    '''
    chi = calculate_chi(present_args, total_area, total_planted_area)
    max_q1 = maximize_output(future_args, total_area, total_planted_area)

    return chi * max_q1

In [82]:
present_args

[[array([100, 100, 100]),
  array([100, 100,   0]),
  array([10, 20, 15]),
  array([40, 70,  0]),
  array([250, 500, 375])],
 [array([100, 100, 100]),
  array([100, 100,   0]),
  array([20, 10, 15]),
  array([60, 30,  0]),
  array([300, 150, 225])]]

In [72]:
present_total_planted_acreage

array([100, 100,   0])

In [73]:
40 * 250 + 70 * 500 + 60*300 + 30 * 150

67500

In [74]:
67500 / (50000 + 37500)

0.7714285714285715

In [79]:
# calculate phi, gamma, chi in the present
f = [calculate_phi(a[0], a[1], a[2], a[3]) for a in present_args]
g = [calculate_gamma([present_args[i][3:] for i in range(len(present_args))])]
x = calculate_chi(present_args, total_acres, present_total_planted_acreage)
print(f, g, x, sep='\n')

[0.9, 0.9090909090909091]
[0.84375]
0.7714285714285715


In [120]:
Q1 = match_moments(present_args, future_args)
Q1

59885.0

In [117]:
Q0 = sum([present_args[i][3] * present_args[i][4]
         for i in range(len(present_args))]).sum()

67500

In [95]:
calculate_welfare_changes(Q0, Q1)

[-114.53520604258404, -144.34114767155702, 29.80594162897299]

# 4. Shift equilibrium and calculate welfare changes

In [111]:
demand_elasticity=-0.1
supply_elasticity=1
ag_GDP=1000

In [112]:
future_args_reallocated = match_moments(present_args, future_args)

In [118]:
Q1

52430.0

In [113]:
present_total_cal, future_total_cal, future_total_cal_reallocated = [
    sum(
        sum(args[i][3] * args[i][4]) for i in range(len(args))) 
    for args in [present_args, future_args, future_args_reallocated]
]

# calculate damages (in calories) with and without reallocation
calorie_damages_no_reallocation = present_total_cal - future_total_cal
calorie_damages_with_reallocation = present_total_cal - future_total_cal_reallocated

# Q0: The inital quantity
Q0 = present_total_cal
# Q1: The quantity considering damages but not price change
Q1 = future_total_cal_reallocated
# Q2: The quantity considering damagaes and price change
Q2 = (
    (supply_elasticity - demand_elasticity)/
    ((supply_elasticity/Q0) - (demand_elasticity/Q1))
)

# percent changes
P0_P1 = (Q2 / Q0 - 1) / demand_elasticity
Q0_Q1 = (Q1 - Q0)/Q0
Q1_Q2 = (Q2 - Q1)/Q1
Q0_Q2 = (Q2 - Q0)/Q0
# areas to calculate: B, C, D, F, G, H
C = 0.5*((Q2 - Q1)/Q0)*P0_P1*ag_GDP
D = 0.5*-(Q2/Q0 - 1)*P0_P1*ag_GDP
F = (Q1/Q0)*(-(Q1/Q0 - 1)/supply_elasticity)*ag_GDP

G = (0.5 * 
     ((Q2 - Q1)/Q0)* # height
     (-(Q1/Q0 - 1)/supply_elasticity + # base 1
     -(Q2/Q0 - 1)/supply_elasticity) * # base 2
    ag_GDP)
H = 0.5*(1 - Q2/Q0)*(-(Q2/Q0 - 1)/supply_elasticity)*ag_GDP

B = P0_P1*(Q2/Q0)*ag_GDP - C
# percent change from P0 to P1
P0_P1 = (Q2/Q0 - 1)/demand_elasticity

In [121]:
# areas to calculate: B, C, D, F, G, H
C = 0.5*((Q2 - Q1)/Q0)*P0_P1*ag_GDP
D = 0.5*-(Q2/Q0 - 1)*P0_P1*ag_GDP
F = (Q1/Q0)*(-(Q1/Q0 - 1)/supply_elasticity)*ag_GDP

G = (0.5 * 
     ((Q2 - Q1)/Q0)* # height
     (-(Q1/Q0 - 1)/supply_elasticity + # base 1
     -(Q2/Q0 - 1)/supply_elasticity) * # base 2
    ag_GDP)
H = 0.5*(1 - Q2/Q0)*(-(Q2/Q0 - 1)/supply_elasticity)*ag_GDP

B = P0_P1*(Q2/Q0)*ag_GDP - C

print(B, C, D, F, G, H, sep='\n')

237.0406214315355
11.121717397390723
3.242250659885824
100.08763237311385
6.039366154861896
0.3242250659885824


In [122]:
# calculate consumer and producer surplus changes
delta_CS = -B - C - D
delta_PS = B - F - G - H
tot = delta_CS + delta_PS

print(delta_CS, delta_PS, tot, sep='\n')

-251.40458948881204
130.58939783757117
-120.81519165124087


In [90]:
def calculate_welfare_changes(Q0, Q1, demand_elasticity=0.1,
                              supply_elasticity=1, ag_GDP=1000):
    '''
    Calculate welfare changes resulting from climate shock to agricultural
    production.
    Parameters
    ----------
    Q0: float
        Total calories produced before the climate shock
    Q1: float
        Total calories produced after the climate shock but before considering
        the equilibrium shift
    '''
    # Calculate Q2: post-climate shock quantity with equilibrium shift
    Q2 = (
        (supply_elasticity - demand_elasticity) /
        ((supply_elasticity / Q0) - (demand_elasticity / Q1))
    )

    # percent change from P0 to P1
    P0_P1 = (Q2 / Q0 - 1) / demand_elasticity

    # calculate areas corresponding to welfare
    # areas to calculate: B, C, D, F, G, H
    C = 0.5 * ((Q2 - Q1) / Q0) * P0_P1 * ag_GDP
    D = 0.5 * -(Q2 / Q0 - 1) * P0_P1 * ag_GDP
    F = (Q1 / Q0) * (-(Q1 / Q0 - 1) / supply_elasticity) * ag_GDP

    G = (0.5 *
         ((Q2 - Q1) / Q0) *  # height
         (-(Q1 / Q0 - 1) / supply_elasticity +  # base 1
          -(Q2 / Q0 - 1) / supply_elasticity) *  # base 2
         ag_GDP)
    H = 0.5 * (1 - Q2 / Q0) * (-(Q2 / Q0 - 1) / supply_elasticity) * ag_GDP

    B = P0_P1 * (Q2 / Q0) * ag_GDP - C

    # calculate consumer and producer surplus changes
    delta_CS = -B - C - D
    delta_PS = B - F - G - H
    total_welfare_change = delta_CS + delta_PS
    return [total_welfare_change, delta_CS, delta_PS]

# Benchmark

In [ ]:
timeit.timeit("lambda: calculate_phi(present_rice_yields, present_rice_acreage, present_total_planted_acreage)")

In [29]:
%timeit -n 100000 lambda: calculate_phi(present_rice_yields, present_rice_acreage, present_total_planted_acreage)

64.9 ns ± 3.23 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [ ]:
%timeit -n 100000 lambda: match_moments(present_args, future_args)